<a id='longitudinal'></a>
## Longitudinal resilience analysis

In [12]:
import os
import pandas as pd
from qiime2 import Visualization
from skbio import OrdinationResults
import matplotlib.pyplot as plt
from seaborn import scatterplot
import numpy as np

import qiime2 as q2

# do not increase this value!
n_jobs = 3
    

%matplotlib inline
data_dir = 'CE'

In [13]:
#only retain features that are present at some minimal frequency (25) and in at least 4 samples
! qiime feature-table filter-features \
--i-table $data_dir/dada2_table_align_filtered.qza \
--p-min-frequency 25 \
--p-min-samples 4 \
--o-filtered-table $data_dir/table_abund254.qza

Saved FeatureTable[Frequency] to: CE/table_abund254.qza


In [14]:
#example: comparing diff. abundance within continents
! qiime feature-table filter-samples \
--i-table $data_dir/table_abund254.qza \
--m-metadata-file $data_dir/food-metadata.tsv \
--p-where "[continent]='North_America' or [continent]='Europe'" \
--o-filtered-table $data_dir/table_abund254_continent.qza

Saved FeatureTable[Frequency] to: CE/table_abund254_continent.qza


In [23]:
# Run the following commands and inspect the results.
# We will look at infants and their mothers' microbiota just prior to birth
! qiime feature-table filter-samples \
    --i-table $data_dir/table_abund254_continent.qza \
    --m-metadata-file $data_dir/food-metadata.tsv \
    --o-filtered-table $data_dir/filtered-table-deblur-with-continent.qza

# Repeat our core-metrics diversity analysis from before, this time with mothers
! qiime diversity core-metrics \
  --i-table $data_dir/filtered-table-deblur-with-continent.qza \
  --m-metadata-file $data_dir/food-metadata.tsv \
  --p-sampling-depth 1800 \
  --p-n-jobs $n_jobs \
  --output-dir $data_dir/core-metrics-with-continent

# this creates an interactive line plot — useful for looking at changes in alpha and beta diversity across time
! qiime longitudinal volatility \
  --m-metadata-file $data_dir/food-metadata.tsv \
  --m-metadata-file $data_dir/core-metrics-with-continent/observed_features_vector.qza \
  --m-metadata-file $data_dir/core-metrics-with-continent/shannon_vector.qza \
  --m-metadata-file $data_dir/core-metrics-with-continent/jaccard_pcoa_results.qza \
  --p-default-group-column 'continent'\
  --p-default-metric 'observed_features' \
  --p-state-column 'longitude' \
  --p-individual-id-column 'continent' \
  --o-visualization $data_dir/core-metrics-with-continent/volatility.qzv



Saved FeatureTable[Frequency] to: CE/filtered-table-deblur-with-continent.qza
Usage: qiime diversity core-metrics [OPTIONS]

  Applies a collection of diversity metrics (non-phylogenetic) to a feature
  table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                       The feature table containing the samples over which
                       diversity metrics should be computed.        [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)     The total frequency that each sample should be
                       rarefied to prior to computing diversity metrics.
                                                                    [required]
  --m-metadata-file METADATA...
    (multiple          The sample metadata to use in the emperor plots.
     arguments will    
     be merged)                                                     [required]
  --p-with-replacement / --p-no-with-replacement
                       Rarefy with replacement by sampling from

In [17]:
Visualization.load(f'{data_dir}/core-metrics-with-continent/jaccard_emperor.qzv')

<visualization: Visualization uuid: 32d546ba-52c6-493b-85e0-aa8607b6a921>

In [24]:
Visualization.load(f'{data_dir}/core-metrics-with-continent/volatility.qzv')

<visualization: Visualization uuid: bb78dab9-1625-42d1-a3e4-a7ebb6137867>